Please refer to my main article (404dataguyblog.wordpress.com/2023/02/09/how-has-the-electorate-changed-since-reagan/) I wrote regarding the Election Trends in Post-Reagan Politics. These notebooks are meant to be a guide to see the analytical process and how to create the visuals and modify data using python. All my data is from avalible free on https://electionlab.mit.edu/, and I am only focusing on Presidental elections. Addtionally, the csv file can be found in my github. 

The first step (after importing our required packages) is to import the data into dataframes, and create a pivot table so can convietally view the results of each state for each election in a single row. This process is done below, taking our 'df_raw' dataframe and using it to create of 'df' dataframe.

In [16]:
import pandas as pd
import numpy as np

In [17]:
df_raw = pd.read_csv('1976-2020-president.csv')
df_raw.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT
1,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"FORD, GERALD",REPUBLICAN,False,504070,1182850,20210113,NaN,REPUBLICAN
2,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"MADDOX, LESTER",AMERICAN INDEPENDENT PARTY,False,9198,1182850,20210113,NaN,OTHER
3,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"BUBAR, BENJAMIN """"BEN""""",PROHIBITION,False,6669,1182850,20210113,NaN,OTHER
4,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"HALL, GUS",COMMUNIST PARTY USE,False,1954,1182850,20210113,NaN,OTHER


In [18]:


df_raw['party_simplified'].value_counts()
df_raw['party_simplified'] = df_raw['party_simplified'].replace('LIBERTARIAN','OTHER')

df = pd.pivot_table(df_raw,values ='candidatevotes',index = ['year','state'], columns = ['party_simplified'], aggfunc = np.sum)

df['OTHER'] = df['OTHER'].fillna(0)

df['TOTAL'] = df['DEMOCRAT'] + df['OTHER'] + df['REPUBLICAN']

df = df.reset_index()
df.head()

party_simplified,year,state,DEMOCRAT,OTHER,REPUBLICAN,TOTAL
0,1976,ALABAMA,659170.0,19610.0,504070.0,1182850.0
1,1976,ALASKA,44058.0,7961.0,71555.0,123574.0
2,1976,ARIZONA,295602.0,28475.0,418642.0,742719.0
3,1976,ARKANSAS,498604.0,1028.0,267903.0,767535.0
4,1976,CALIFORNIA,3742284.0,179242.0,3882244.0,7803770.0


Once this is done we are now going to calculate the percentages that each party won from the total count values. I also decided to create a seperate datafram that looks at the elections from the national level and called it 'df_pop_vote.' 
To calculate the percentages I create a function (get_percentages) that I will on both the national dataset as well as the state level dataset. I will then export this data into csv files. 

In [19]:
df_pop_vote = df.groupby(['year']).sum()
df_pop_vote


party_simplified,DEMOCRAT,OTHER,REPUBLICAN,TOTAL
year,,,,
1976,40680446.0,2050005.0,38870893.0,81601344.0
1980,35480948.0,7373264.0,43642639.0,86496851.0
1984,37449813.0,1038219.0,54166829.0,92654861.0
1988,41716679.0,1227506.0,48642640.0,91586825.0
1992,44856747.0,20944120.0,38798913.0,104599780.0
1996,47295351.0,10090770.0,39003697.0,96389818.0
2000,50830580.0,4452030.0,50311372.0,105593982.0
2004,58894561.0,1582178.0,61872711.0,122349450.0
2008,69338846.0,2466572.0,59613835.0,131419253.0


In [20]:
def get_percentages(data):
    return_data = data.copy()
    for i in ['DEMOCRAT','REPUBLICAN','OTHER']: 
        col = i + '_%'
        return_data[col] = data[i]/data['TOTAL']
    return return_data

In [21]:
df = get_percentages(df)
df_pop_vote = get_percentages(df_pop_vote)
df_pop_vote = df_pop_vote.reset_index()

In [22]:
df['Winner'] = np.where(df['DEMOCRAT']>df['REPUBLICAN'], 'DEMOCRAT','REPUBLICAN')
df['Winner_%'] = np.where(df['DEMOCRAT']>df['REPUBLICAN'], df['DEMOCRAT_%'],df['REPUBLICAN_%'])
df['Loser_%'] = np.where(df['DEMOCRAT']<df['REPUBLICAN'], df['DEMOCRAT_%'],df['REPUBLICAN_%'])
df.head()

party_simplified,year,state,DEMOCRAT,OTHER,REPUBLICAN,TOTAL,DEMOCRAT_%,REPUBLICAN_%,OTHER_%,Winner,Winner_%,Loser_%
0,1976,ALABAMA,659170.0,19610.0,504070.0,1182850.0,0.557273,0.426149,0.016579,DEMOCRAT,0.557273,0.426149
1,1976,ALASKA,44058.0,7961.0,71555.0,123574.0,0.356531,0.579046,0.064423,REPUBLICAN,0.579046,0.356531
2,1976,ARIZONA,295602.0,28475.0,418642.0,742719.0,0.398000,0.563661,0.038339,REPUBLICAN,0.563661,0.398000
3,1976,ARKANSAS,498604.0,1028.0,267903.0,767535.0,0.649617,0.349043,0.001339,DEMOCRAT,0.649617,0.349043
4,1976,CALIFORNIA,3742284.0,179242.0,3882244.0,7803770.0,0.479548,0.497483,0.022969,REPUBLICAN,0.497483,0.479548


In [23]:
df.to_csv('pres_election.csv',index = False)
df_pop_vote.to_csv('pres_pop_vote.csv',index = False)
df

party_simplified,year,state,DEMOCRAT,OTHER,REPUBLICAN,TOTAL,DEMOCRAT_%,REPUBLICAN_%,OTHER_%,Winner,Winner_%,Loser_%
0,1976,ALABAMA,659170.0,19610.0,504070.0,1182850.0,0.557273,0.426149,0.016579,DEMOCRAT,0.557273,0.426149
1,1976,ALASKA,44058.0,7961.0,71555.0,123574.0,0.356531,0.579046,0.064423,REPUBLICAN,0.579046,0.356531
2,1976,ARIZONA,295602.0,28475.0,418642.0,742719.0,0.398000,0.563661,0.038339,REPUBLICAN,0.563661,0.398000
3,1976,ARKANSAS,498604.0,1028.0,267903.0,767535.0,0.649617,0.349043,0.001339,DEMOCRAT,0.649617,0.349043
4,1976,CALIFORNIA,3742284.0,179242.0,3882244.0,7803770.0,0.479548,0.497483,0.022969,REPUBLICAN,0.497483,0.479548
...,...,...,...,...,...,...,...,...,...,...,...,...
607,2020,VIRGINIA,2413568.0,84526.0,1962430.0,4460524.0,0.541095,0.439955,0.018950,DEMOCRAT,0.541095,0.439955
608,2020,WASHINGTON,2369612.0,133368.0,1584651.0,4087631.0,0.579703,0.387670,0.032627,DEMOCRAT,0.579703,0.387670
609,2020,WEST VIRGINIA,235984.0,13286.0,545382.0,794652.0,0.296965,0.686316,0.016719,REPUBLICAN,0.686316,0.296965
610,2020,WISCONSIN,1630866.0,56991.0,1610184.0,3298041.0,0.494495,0.488224,0.017280,DEMOCRAT,0.494495,0.488224


As you we can, we have successfully exported our data into csv files. We can follow a similiar process for the more granular county level data. However, some of the states in 2020 created distinct rows for different "modes" of voting (such aselection day, early voting), etc. In order to handle this we first seperate the 2020 data, group by county, and combine it to our remaining election data. We will reuse the get_percentages function. 

In [29]:
df_raw = pd.read_csv('countypres_2000-2020.csv')
df_raw['party'] = df_raw['party'].replace(['LIBERTARIAN','GREEN'],'OTHER')
df_2020 = df_raw[df_raw['year']==2020]
df_raw = df_raw[df_raw['mode']=='TOTAL']

print(df_raw.columns)

df = pd.pivot_table(df_raw,values ='candidatevotes',index = ['year','state','county_name'], columns = ['party'], aggfunc = np.sum)

df['OTHER'] = df['OTHER'].fillna(0)

df['TOTAL'] = df['DEMOCRAT'] + df['OTHER'] + df['REPUBLICAN']

df = df.reset_index()
df.head()

Index(['year', 'state', 'state_po', 'county_name', 'county_fips', 'office',
       'candidate', 'party', 'candidatevotes', 'totalvotes', 'version',
       'mode'],
      dtype='object')


party,year,state,county_name,DEMOCRAT,OTHER,REPUBLICAN,TOTAL
0,2000,ALABAMA,AUTAUGA,4942,273,11993,17208
1,2000,ALABAMA,BALDWIN,13997,1611,40872,56480
2,2000,ALABAMA,BARBOUR,5188,111,5096,10395
3,2000,ALABAMA,BIBB,2710,118,4273,7101
4,2000,ALABAMA,BLOUNT,4977,329,12667,17973


In [30]:
df_2020[df_2020['mode']!='TOTAL']

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
50930,2020,ARIZONA,AZ,APACHE,4001.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,16460,35172,20220315,EARLY VOTE
50931,2020,ARIZONA,AZ,APACHE,4001.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,6539,35172,20220315,ELECTION DAY
50932,2020,ARIZONA,AZ,APACHE,4001.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,294,35172,20220315,PROVISIONAL
50933,2020,ARIZONA,AZ,APACHE,4001.0,US PRESIDENT,OTHER,OTHER,0,35172,20220315,EARLY VOTE
50934,2020,ARIZONA,AZ,APACHE,4001.0,US PRESIDENT,OTHER,OTHER,0,35172,20220315,ELECTION DAY
...,...,...,...,...,...,...,...,...,...,...,...,...
71965,2020,VIRGINIA,VA,WINCHESTER CITY,51840.0,US PRESIDENT,OTHER,OTHER,24,12113,20220315,ELECTION DAY
71966,2020,VIRGINIA,VA,WINCHESTER CITY,51840.0,US PRESIDENT,OTHER,OTHER,0,12113,20220315,PROVISIONAL
71967,2020,VIRGINIA,VA,WINCHESTER CITY,51840.0,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,2825,12113,20220315,ABSENTEE
71968,2020,VIRGINIA,VA,WINCHESTER CITY,51840.0,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,2370,12113,20220315,ELECTION DAY


In [31]:
df_2020 = df_2020.groupby(['year', 'state', 'state_po', 'county_name', 'county_fips', 'office',
       'candidate', 'party', 'totalvotes']).sum()


df_2020 = pd.pivot_table(df_2020,values ='candidatevotes',index = ['year','state','county_name'], columns = ['party'], aggfunc = np.sum)

df_2020['OTHER'] = df_2020['OTHER'].fillna(0)

df_2020['TOTAL'] = df_2020['DEMOCRAT'] + df_2020['OTHER'] + df_2020['REPUBLICAN']

df_2020 = df_2020.reset_index()
df = pd.concat([df,df_2020])
df = df.reset_index()
df.tail()

party,index,year,state,county_name,DEMOCRAT,OTHER,REPUBLICAN,TOTAL
21218,3148,2020,WYOMING,SWEETWATER,3823,646,12229,16698
21219,3149,2020,WYOMING,TETON,9848,598,4341,14787
21220,3150,2020,WYOMING,UINTA,1591,372,7496,9459
21221,3151,2020,WYOMING,WASHAKIE,651,136,3245,4032
21222,3152,2020,WYOMING,WESTON,360,93,3107,3560


In [32]:
df = get_percentages(df)
df = df.drop(['index'], axis=1)
df = df.drop_duplicates(subset = ['state','county_name','year'], keep = 'first').reset_index(drop = True)
df

party,year,state,county_name,DEMOCRAT,OTHER,REPUBLICAN,TOTAL,DEMOCRAT_%,REPUBLICAN_%,OTHER_%
0,2000,ALABAMA,AUTAUGA,4942,273,11993,17208,0.287192,0.696943,0.015865
1,2000,ALABAMA,BALDWIN,13997,1611,40872,56480,0.247822,0.723654,0.028523
2,2000,ALABAMA,BARBOUR,5188,111,5096,10395,0.499086,0.490236,0.010678
3,2000,ALABAMA,BIBB,2710,118,4273,7101,0.381636,0.601746,0.016617
4,2000,ALABAMA,BLOUNT,4977,329,12667,17973,0.276915,0.704779,0.018305
...,...,...,...,...,...,...,...,...,...,...
18914,2020,VIRGINIA,WILLIAMSBURG CITY,4790,137,1963,6890,0.695210,0.284906,0.019884
18915,2020,VIRGINIA,WINCHESTER CITY,6610,282,5221,12113,0.545695,0.431025,0.023281
18916,2020,VIRGINIA,WISE,3110,139,13366,16615,0.187180,0.804454,0.008366
18917,2020,VIRGINIA,WYTHE,3143,197,11733,15073,0.208519,0.778412,0.013070


In [33]:
df.to_csv('pres_election_county.csv',index = False)

In [34]:
df

party,year,state,county_name,DEMOCRAT,OTHER,REPUBLICAN,TOTAL,DEMOCRAT_%,REPUBLICAN_%,OTHER_%
0,2000,ALABAMA,AUTAUGA,4942,273,11993,17208,0.287192,0.696943,0.015865
1,2000,ALABAMA,BALDWIN,13997,1611,40872,56480,0.247822,0.723654,0.028523
2,2000,ALABAMA,BARBOUR,5188,111,5096,10395,0.499086,0.490236,0.010678
3,2000,ALABAMA,BIBB,2710,118,4273,7101,0.381636,0.601746,0.016617
4,2000,ALABAMA,BLOUNT,4977,329,12667,17973,0.276915,0.704779,0.018305
...,...,...,...,...,...,...,...,...,...,...
18914,2020,VIRGINIA,WILLIAMSBURG CITY,4790,137,1963,6890,0.695210,0.284906,0.019884
18915,2020,VIRGINIA,WINCHESTER CITY,6610,282,5221,12113,0.545695,0.431025,0.023281
18916,2020,VIRGINIA,WISE,3110,139,13366,16615,0.187180,0.804454,0.008366
18917,2020,VIRGINIA,WYTHE,3143,197,11733,15073,0.208519,0.778412,0.013070


 now have our data cleaned and in a more usable format. Continue to part 2 to see the EDA and metric creation process.  